In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re
import os
import sys

from google.cloud import bigquery
import pyarrow
from datetime import datetime
from tqdm import tqdm

from oauth2client.client import GoogleCredentials
from google.colab import auth, files

import ast
import json
import re
import random

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/content/drive/Shareddrives/SIADS - 694-695 Team Drive/Notebooks/compact-scene-317315-56e479e9e148.json'

In [6]:
def extract_on_dates(startdate, enddate, filename):
  import os.path
  
  bigquery_client = bigquery.Client()

  dates = pd.date_range(startdate, enddate,)
  formatted_dates = [date.strftime('%Y%m%d') for date in dates]
  dataset = pd.DataFrame()

  for idx, date in enumerate(formatted_dates):
    query = f'SELECT * FROM bigquery-public-data.google_analytics_sample.ga_sessions_{date}'
    if idx % 5 == 0: print(date)
    df = bigquery_client.query(query).to_dataframe()
    dataset = pd.concat([dataset,df], sort=False)

  if os.path.isfile(filename):
    append_csv_rows(filename, dataset)
  else:
    dataset.to_csv(filename, index=False, header=True)

  return print(f'{startdate} to {enddate} data has been added to filename provided!')

def append_csv_rows(filename, dataframe):
  return dataframe.to_csv(filename, mode='a', index=False, header=False)

def get_selected_columns(filename='/content/drive/Shareddrives/SIADS - 694-695 Team Drive/Notebooks/final_columns.txt'):
  """
  filename: """
  columns = []
  with open(filename, 'r') as f:
    for ln in f.readlines():
      col = (ln.strip().split('\xa0\xa0')[1])
      groups = re.search(r"'(\w.+)'", col)
      columns.append(groups.group(1))
  return columns

def fileCount(filename):
  n = sum(1 for line in open(filename)) 
  return n

def append_sample(filename):
  n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
  s = int(n * .1) #desired sample size
  skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
  df = pd.read_csv(filename, skiprows=skip)
  return df

def get_full_sample(filenames: list):
  fullSample = pd.DataFrame()

  for file in filenames:
    if len(fullSample) == 0:
      fullSample = append_sample(file)
    else:
      fileSample = append_sample(file)
      fullSample = pd.concat([fullSample, fileSample])

  return fullSample

In [8]:
filename1 = '/content/drive/Shareddrives/SIADS - 694-695 Team Drive/datasets/monthly_partitioned_data/August 2016 Google Analytics Dataset.csv'
filename2 = '/content/drive/Shareddrives/SIADS - 694-695 Team Drive/datasets/monthly_partitioned_data/September 2016 Google Analytics Dataset.csv'
filename3 = '/content/drive/Shareddrives/SIADS - 694-695 Team Drive/datasets/monthly_partitioned_data/October 2016 Google Analytics Dataset.csv'
filename4 = '/content/drive/Shareddrives/SIADS - 694-695 Team Drive/datasets/monthly_partitioned_data/November 2016 Google Analytics Dataset.csv'
filename5 = '/content/drive/Shareddrives/SIADS - 694-695 Team Drive/datasets/monthly_partitioned_data/December 2016 Google Analytics Dataset.csv'
filename6 = '/content/drive/Shareddrives/SIADS - 694-695 Team Drive/datasets/monthly_partitioned_data/January 2017 Google Analytics Dataset.csv'

In [ ]:
extraction_dates = [('08-01-2016','08-31-2016', filename1),
                    ('09-01-2016','09-30-2016', filename2),
                    ('10-01-2016','10-31-2016', filename3),
                    ('11-01-2016','11-30-2016', filename4),
                    ('12-01-2016','12-31-2016', filename5),
                    ('01-01-2017','01-31-2017', filename6)]
for start, end, filename in extraction_dates:
  extract_on_dates(start, end, filename)

In [9]:
files = [filename1, filename2, filename3, filename4, filename5, filename6]
count = 0
for f in files:
  count += fileCount(f)
print("Total row count is: ", count)

Total row count is:  501093


In [ ]:
keep_cols = set()

for col in get_selected_columns():
  if col != 'clientId': keep_cols.add(col.split('.')[0])

In [ ]:
sample = get_full_sample([filename1, filename2, filename3, filename4,filename5, filename6])[list(keep_cols)]

In [ ]:
len(sample)

50106

In [ ]:
sample.to_csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/sample_file.csv',index=False,header=True)